# RNN Language Model

### Data Things
We're using SST once again. Load the data!

In [1]:
sst_home = '../data/trees'

import re
import random

# Let's do 2-way positive/negative classification instead of 5-way
easy_label_map = {0:0, 1:0, 2:None, 3:1, 4:1}

def load_sst_data(path):
    data = []
    with open(path) as f:
        for i, line in enumerate(f): 
            example = {}
            
            # Strip out the parse information and the phrase labels---we don't need those here
            text = re.sub(r'\s*(\(\d)|(\))\s*', '', line)
            example['text'] = text[1:]
            data.append(example)

    random.seed(1)
    random.shuffle(data)
    return data
     
training_set = load_sst_data(sst_home + '/train.txt')
dev_set = load_sst_data(sst_home + '/dev.txt')
test_set = load_sst_data(sst_home + '/test.txt')

To simplify your implementation, we'll use a fixed unrolling length of 20. This means that we'll have to expand each sentence into a sequence of 21 word indices. In the conversion process, we'll mark the start of each sentence with a special word symbol `<S>`, mark the end of each sentence (if it occurs within the first 21 words) with a special word symbol `</S>`, mark extra tokens after `</S>` with a special word symbol `<PAD>`, and mark out-of-vocabulary words with `<UNK>`, for unknown. As in the previous assignment, we'll use a very small vocabulary for this assignment, so you'll see `<UNK>` often.

In [2]:
import collections
import torch
import numpy as np

max_seq_length = 20

def sentence_to_padded_index_sequence(datasets):
    '''Annotates datasets with feature vectors.'''
    
    START = "<S>"
    END = "</S>"
    END_PADDING = "<PAD>"
    UNKNOWN = "<UNK>"
    SEQ_LEN = max_seq_length + 1
    
    # Extract vocabulary
    def tokenize(string):
        return string.lower().split()
    
    word_counter = collections.Counter()
    for example in datasets[0]:
        word_counter.update(tokenize(example['text']))
    
    # Only enter word into vocabulary if it appears > 25 times. Add special symbols to vocabulary.
    vocabulary = set([word for word in word_counter if word_counter[word] > 25]) 
    vocabulary = list(vocabulary)
    vocabulary = [START, END, END_PADDING, UNKNOWN] + vocabulary
        
    word_indices = dict(zip(vocabulary, range(len(vocabulary))))
    indices_to_words = {v: k for k, v in word_indices.items()}
        
    for i, dataset in enumerate(datasets):
        for example in dataset:
            example['index_sequence'] = torch.zeros((SEQ_LEN))
            
            token_sequence = [START] + tokenize(example['text']) + [END]

            for i in range(SEQ_LEN):
                if i < len(token_sequence):
                    if token_sequence[i] in word_indices:
                        index = word_indices[token_sequence[i]]
                    else:
                        index = word_indices[UNKNOWN]
                else:
                    index = word_indices[END_PADDING]
                example['index_sequence'][i] = index
                
            example['target_sequence'] = example["index_sequence"][1:]
            example['index_sequence'] = example['index_sequence'].long().view(1,-1)
            example['target_sequence'] = example['target_sequence'].long().view(1,-1)
            
    return indices_to_words, word_indices
    
indices_to_words, word_indices = sentence_to_padded_index_sequence([training_set, dev_set, test_set])

Let's batchify data and define the evaluation metric,

In [3]:
# This is the iterator we'll use during training. 
# It's a generator that gives you one batch at a time.
def data_iter(source, batch_size):
    dataset_size = len(source)
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while True:
        start += batch_size
        if start > dataset_size - batch_size:
            # Start another epoch.
            start = 0
            random.shuffle(order)   
        batch_indices = order[start:start + batch_size]
        batch = [source[index] for index in batch_indices]
        yield [source[index] for index in batch_indices]

# The following function gives batches of vectors and labels, 
# these are the inputs to your model and loss function
def get_batch(batch):
    vectors = []
    targets = []
    for d in batch:
        vectors.append(d["index_sequence"])
        targets.append(d["target_sequence"])
    return vectors, targets


'\ndef evaluate(model, criterion, data_iter):\n    model.eval()\n    total_loss = 0\n    for i in range(len(data_iter)):\n        inputs, targets = get_batch(data_iter[i])\n        inputs = Variable(torch.stack(inputs).squeeze(), volatile=True) # batch_size, seq_len\n        targets = torch.stack(targets).squeeze().view(-1) # flattened\n\n        hidden, c_t = model.init_hidden()\n        hiddens, output = model(inputs, hidden, c_t)\n        \n        output_flat = torch.stack(output, dim=2).view(-1, vocab_size)\n        total_loss += len(inputs) * criterion(output_flat, targets).data\n      \n    return total_loss[0] / len(data_iter)\n'

### Model time!

Now, using the starter code and hyperparameter values provided below, implement an LSTM language model with dropout on the non-recurrent connections. `lstm_step` is provided for you, you need to complete the forward function by implementing dropout and using a decoder. 

We won't be evaluating our model in the conventional way (perplexity on a held-out test set) for a few reasons: to save time, because we have no baseline to compare against, and because overfitting the training set is a less immediate concern with these models than it was with sentence classifiers. Instead, we'll use the value of the loss function to make sure that the model is converging as expected, and we'll use samples drawn from the model to qualitatively evaluate it.

So you will also need to complete the `sample` function such that when it is called, and it is provided with the starting symbol `<S>`, then is generates a sentence that is 20 tokens long (20 being our max sequence length). You should calucalte word weights with a softmax but make sure to use the softmax trick of subtracting the maximum value,
$$ weight_i = \frac{\exp(x_i - \max_i(x))}{\sum_i(\exp(x_i - max_i(x)))} $$
Then use these weights to sample from the distribution and generate tokens.

Hint: you will need to use `torch.multinomial()` 

In [54]:
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

class RNNLM(nn.Module):
    def __init__(self, vocab_size, seq_len, embedding_dim, hidden_size, batch_size):
        super(RNNLM, self).__init__()
        
        self.encoder = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.embedding_size = embedding_dim
        self.hidden_size = hidden_size
        self.vocab_size = vocab_size
        self.batch_size = batch_size
        self.seq_len = seq_len
        
        self.linear_f = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.linear_i = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.linear_ctilde = nn.Linear(embedding_dim + hidden_size, hidden_size)
        self.linear_o = nn.Linear(embedding_dim + hidden_size, hidden_size)

        """
        Your code goes here.
        
        Define missing parameters
        """
        
        self.init_weights()
           
    def forward(self, x, hidden, c):
        x_emb = self.encoder(x)
        embs = torch.chunk(x_emb, x_emb.size()[1], 1)  
        hiddens = []
        logits = []
        for i in range(self.seq_len - 1):
            hidden, c = self.lstm_step(embs[i].squeeze(), hidden, c)
            """
            Your code goes here.
            """
            
        return    
        
    def sample(self, x_start, hidden, c):
        indices = [int(x_start.data.numpy())]
        for i in range(self.seq_len - 1):
            x_emb = self.encoder(x_start)
            hidden, c = self.lstm_step(x_emb.view(1,-1), hidden, c) 
            
            """
            Your code goes here.
            
            This function returns a list of word indices.
            In the training loop, we use these indices to lookup 
            the word from indices_to_words
            """
            
        return indices
        
    def lstm_step(self, emb, hid, c_t):
        emb_drop = self.dropout(emb)
        combined = torch.cat((hid, emb_drop), 1)
        f = F.sigmoid(self.linear_f(combined))
        i = F.sigmoid(self.linear_i(combined))
        c_tilde = F.tanh(self.linear_ctilde(combined))
        c_t = f * c_t + i * c_tilde
        o = F.sigmoid(self.linear_o(combined))
        hid = o * F.tanh(c_t)
        return hid, c_t
    
    def init_weights(self):
        initrange = 0.1
        lin_layers = [self.linear_f, self.linear_i, self.linear_ctilde, self.linear_o, self.decoder]
        em_layer = [self.encoder]
     
        for layer in lin_layers+em_layer:
            layer.weight.data.uniform_(-initrange, initrange)
            if layer in lin_layers:
                layer.bias.data.fill_(0)
    
    def init_hidden(self, bsz):
        h0 = Variable(torch.zeros(bsz, self.hidden_size))
        c0 = Variable(torch.zeros(bsz, self.hidden_size))
        return h0, c0

### Define the training loop,

In [55]:
def training_loop(batch_size, vocab_size, num_epochs, display_freq, model, criterion, optim, training_iter):
    step = 1
    epoch = 1
    total_batches = int(len(training_set) / batch_size)

    while epoch <= num_epochs:
        model.train()      
        inputs, targets = get_batch(next(training_iter)) 
        inputs = Variable(torch.stack(inputs).squeeze(), volatile=False) # batch_size,  seq_len
        targets = Variable(torch.stack(targets).squeeze().view(-1)) # batch_size,seq_len --> flat, b*s
    
        model.zero_grad()     
        hidden, c_t = model.init_hidden(batch_size)
        hiddens, output = model(inputs, hidden, c_t)
        output_flat = torch.stack(output, dim=1).view(-1, vocab_size) # batch_size, seq_len, hid --> b*s,h

        lossy = criterion(output_flat, targets)
        lossy.backward()
        optim.step()
        
        if step % total_batches == 0:
            if epoch % display_freq == 0:
                hidden, c_t = model.init_hidden(1)
                input = Variable(torch.zeros(1, 1).long(), volatile=True)
                indices = model.sample(input, hidden, c_t)
                words = [indices_to_words[index] for index in indices]
                print("Epoch: {}; Loss: {}; Sample: {}".format(epoch, lossy.data[0], ' '.join(words)))
            epoch += 1
        step += 1

### Train and sample!

Now train model with the suggested hyperparameters.

Once you're confident your model is doing what you want, let it run for the full 350 epochs. This will take some time—likely between five and thirty minutes, so do it at home if you don't have time during the class. We train the model for a fairly long time because these small improvements in cost correspond to fairly large improvements in sample quality.

Samples from a trained models should have coherent portions, but they will not resemble interpretable English sentences.

In [ ]:
# Hyper Parameters 
vocab_size = len(indices_to_words)
seq_length = max_seq_length + 1
hidden_dim = 32
embedding_dim = 16
batch_size = 256
learning_rate = 0.3
num_epochs = 350
display_epoch_freq = 1

# Build, initialize, and train model
lm = RNNLM(vocab_size, seq_length, embedding_dim, hidden_dim, batch_size)
lm.init_weights()

# Loss and Optimizer
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(lm.parameters(), lr=learning_rate)

# Train the model
training_iter = data_iter(training_set, batch_size)
training_loop(batch_size, vocab_size, num_epochs, display_epoch_freq, lm, loss, optimizer, training_iter)

## Questions:

**Question 1:** Looking at the samples that your model produced towards the end of training, what properties of (written) English does it seem to have learned.

**Question 2:** If we could make the model as big as we wanted, train as long as we wanted, and adjust or remove dropout at will, could we ever get the model to reach a cost value of 0.0?

**Question 3:** Would the model be any worse if we were to just delete unknown words instead of using an `<UNK>` token?

## Extras:

* Do weight tying, i.e. reuse the input word embedding matrix as the output classification matrix. _https://arxiv.org/pdf/1611.01462.pdf_
* Implement beam search decoding